Задача A

Для клиентов, у которых неизвестен пол (которых нет в обучающей выборке customers_gender_train.csv, но которые есть в transactions.csv), необходимо предсказать вероятность быть мужского пола (значение 1).

In [30]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

Читаем входные файлы с данными

In [31]:
transactions = pd.read_csv('transactions.csv')
customers_gender = pd.read_csv('customers_gender_train.csv')

In [32]:
customers_gender.head()

,customer_id,gender
0,75562265,0
1,10928546,1
2,69348468,1
3,84816985,1
4,61009479,0


In [33]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


Для каждого клиента подсчитываем различные mcc_code

In [34]:
X = transactions.groupby('customer_id') \
                    .apply(lambda x: x[['mcc_code']].unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

In [35]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [36]:
customers_gender = customers_gender.set_index('customer_id')

In [37]:
customers_gender.head()

,gender
customer_id,
75562265,0
10928546,1
69348468,1
84816985,1
61009479,0


In [38]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [39]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [46]:
clf = GradientBoostingClassifier(random_state=13, verbose=1, n_estimators=700, max_depth=2, criterion='mse' )
clf.fit(X_train, Y_train.values[:, 0])

      Iter       Train Loss   Remaining Time 
         1           1.3443           23.98s
         2           1.3209           22.03s
         3           1.2991           21.69s
         4           1.2815           21.21s
         5           1.2649           21.67s
         6           1.2515           20.81s
         7           1.2384           20.67s
         8           1.2270           21.00s
         9           1.2166           20.82s
        10           1.2073           20.34s
        20           1.1400           19.12s
        30           1.0959           18.35s
        40           1.0640           17.81s
        50           1.0381           17.44s
        60           1.0171           17.00s
        70           0.9999           16.49s
        80           0.9848           16.25s
        90           0.9721           15.86s
       100           0.9603           15.45s
       200           0.8862           12.59s
       300           0.8459            9.85s
       40

GradientBoostingClassifier(criterion='mse', init=None, learning_rate=0.1,
              loss='deviance', max_depth=2, max_features=None,
              max_leaf_nodes=None, min_impurity_split=1e-07,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=700,
              presort='auto', random_state=13, subsample=1.0, verbose=1,
              warm_start=False)




Предсказываем пол для тестовых данных и создаём файл с ответом

In [41]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [42]:
result.to_csv('baseline_a.csv', index=False)

-